In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import joblib
import os
import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from pymongo import MongoClient
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,classification_report


#### 1.) 실시간 사출 + PLC데이터 수집 함수 작성 
modelBasePath = "PYAPI/LogicLayer/Models/Neonent"
def get_mongo_data(machine_number,plc_length,test_length):
    serverAddress = "private_server_address"
    clientUpdate = MongoClient("mongodb://private_company_client@{0}/admin".format(serverAddress))
    client = clientUpdate
    mydb = client["privateDB"]
    mycoll = mydb["neonentplcdata"]
    plc_data = pd.DataFrame(list(mycoll.find({"LineName": str(machine_number), "$or": [{"StartAddress": "3000"}, {"StartAddress": "3100"}]}).sort('CreateDate',-1).limit(plc_length))).sort_values(by='CreateDate')
    mycoll1 = mydb["injectionmoldData"]
    test_data = pd.DataFrame(list(mycoll1.find({"PlantNo": f"A01-0{str(machine_number)}"}).sort('CreateDate',-1).limit(test_length))).sort_values(by='CreateDate')
    return test_data,plc_data 
test_data, plc = get_mongo_data(machine_number,plc_length,test_length)
    
    
#### 2.) 사출 + PLC시계열 데이터 매칭 전처리 
def preprocessing(test_data, plc): 
    i = 0
    index_plc = []
    index_data = []
    matched_df = pd.DataFrame()
    total_df = pd.DataFrame()
    #### 1.) 사출기별 데이터 생산 시간 고려해서 사출 +PLC데이터 매칭 시도 
    for n, i in tqdm(enumerate(range(len(test_data)))):
        length = 0
        if machine_number == 13:
            matched_plc = plc[(plc.CreateDate <= test_data.CreateDate.iloc[i]) & (plc.CreateDate > test_data.CreateDate.iloc[i] - datetime.timedelta(seconds=50))]  
            length = len(matched_plc)
        elif machine_number == 14:
            matched_plc = plc[(plc.CreateDate <= test_data.CreateDate.iloc[i]) & (plc.CreateDate > test_data.CreateDate.iloc[i] - datetime.timedelta(seconds=80))]  
            length = len(matched_plc)
        elif machine_number == 15:
            matched_plc = plc[(plc.CreateDate <= test_data.CreateDate.iloc[i]) & (plc.CreateDate > test_data.CreateDate.iloc[i] - datetime.timedelta(seconds=55))]  
            length = len(matched_plc)
        elif machine_number == 16:
            matched_plc = plc[(plc.CreateDate <= test_data.CreateDate.iloc[i]) & (plc.CreateDate > test_data.CreateDate.iloc[i] - datetime.timedelta(seconds=50))]  
            length = len(matched_plc)
        elif machine_number == 17:
            matched_plc = plc[(plc.CreateDate <= test_data.CreateDate.iloc[i]) & (plc.CreateDate > test_data.CreateDate.iloc[i] - datetime.timedelta(seconds=50))]  
            length = len(matched_plc)
        elif machine_number == 18:
            matched_plc = plc[(plc.CreateDate <= test_data.CreateDate.iloc[i]) & (plc.CreateDate > test_data.CreateDate.iloc[i] - datetime.timedelta(seconds=60))]  
            length = len(matched_plc)
        if length != 0: 
            total_df = pd.concat([total_df, test_data.iloc[[i]]]) 
            matched_df = pd.concat([matched_df, matched_plc]) 
            index_data.append(n)
            for k in range(length):  
                index_plc.append(n) 

    #### 2.) 실시간 수집 사출 + PLC 매칭 실패 시 사출데이터만 활용 
    if len(index_plc) == 0:
        print('Non Data Are Matched')
        print('Use only Injection Mold Data for Prediction')
        test_data = get_mongo_data(machine_number,plc_length,test_length)[0] 
        for col in test_data.columns:
            try : test_data[col] = test_data[col].astype('float')
            except: pass
        if 'FaultyType' in test_data.columns: 
            test_data = test_data.drop(['FaultyType'],axis=1)  
        else : 
            pass
        test_data = test_data.rename(columns = {'Value1' : 'WaxTank_Temperature','Value2' : 'Injection_Temperature','Value3' : 'CirculationHose_Temperature','Value4' : 'Nozzle_Temperature'})
        test_data = test_data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate'],axis=1)
        test_data['PassOrFail'] = test_data['PassOrFail'].replace({"Y":0, "N":1})
        Label = test_data['PassOrFail'].replace({0:1, 1:0}) 
        Min = test_data.공
    else :
        #### 3.) 실시간 수집 사출 + PLC 매칭 성공 & PLC통계량 파생변수 계산 
        print('Matched Data Exists')
        print('Used Matched Injection Mold & PLC Data for Prediction')
        matched_df['index1'] = index_plc            
        total_df['PassOrFail'] = total_df['PassOrFail'].replace({"Y":0, "N":1})
        total_df['index1'] = index_data  
        total = pd.merge(total_df,matched_df, how='left',on = 'index1') 
        for col in total.columns:
            try : total[col] = total[col].astype('float')
            except: pass 
        total = total.rename(columns = {'Value1' : 'WaxTank_Temperature','Value2' : 'Injection_Temperature','Value3' : 'CirculationHose_Temperature','Value4' : 'Nozzle_Temperature','InjectionTemperature':'PLC_InjectionTemperature','NozzleTemperature':'PLC_NozzleTemperature','CoolingTime':'PLC_CoolingTime','InjectionPressure':'PLC_InjectionPressure','MultiTankTemperature':'PLC_MultiTankTemperature','CirculationTank':'PLC_CirculationTank'}) 
        total = total.rename(columns = {'_id_x' :'PLC_id','CreateDate_x' :'PLC_CreateDate','_id_y' :'_id','CreateDate_y' :'CreateDate'})
        total = total[['YMdate','CreateDate','_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','Aqty','WaxTank_Temperature','Injection_Temperature','CirculationHose_Temperature','Nozzle_Temperature','PLC_CreateDate','PLC_id','StartAddress','LineName','StartEnd','Count','PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','index1','PassOrFail']]
        total['YMdate'] = total['YMdate'].str.replace('오전', 'AM')
        total['YMdate'] = total['YMdate'].str.replace('오후', 'PM')
        df = total['YMdate'].str.split(" ",expand=True).rename(columns = {0 : 'Date',1 : 'AM/PM',2:'Time'})
        total['YMdate'] = pd.to_datetime(df['Date'] + ' ' + df['Time']  + ' ' + df['AM/PM'])
        if machine_number == 13:
            df_mean = total.groupby('index1').mean()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','PassOrFail']]
            df_mean = df_mean.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mean','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mean','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mean','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mean','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mean','PLC_CirculationTank' : 'PLC_CirculationTank_Mean'})
            df_Std = df_mean
            df_min = total.groupby('index1').min()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_min = df_min.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Min','PLC_InjectionPressure' : 'PLC_InjectionPressure_Min','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Min','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Min','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Min','PLC_CirculationTank' : 'PLC_CirculationTank_Min'})
            df_max = total.groupby('index1').max()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_max = df_max.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Max','PLC_InjectionPressure' : 'PLC_InjectionPressure_Max','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Max','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Max','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Max','PLC_CirculationTank' : 'PLC_CirculationTank_Max'})
            df_lower_68 = df_mean - 1.65*df_Std/np.sqrt(len(df_mean))
            df_lower_68 = df_lower_68.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower68','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower68','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower68','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower68','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower68','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower68'})
            df_upper_68 = df_mean + 1.65*df_Std/np.sqrt(len(df_mean))
            df_upper_68 = df_upper_68.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper68','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper68','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper68','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper68','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper68','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper68'})
            df_lower_95 = df_mean - 1.96*df_Std/np.sqrt(len(df_mean))
            df_lower_95 = df_lower_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower95'})
            df_upper_95 = df_mean + 1.96*df_Std/np.sqrt(len(df_mean))
            df_upper_95 = df_upper_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper95'})
            df_lower_99 = df_mean - 2.58*df_Std/np.sqrt(len(df_mean))
            df_lower_99 = df_lower_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower99'})
            df_upper_99 = df_mean + 2.58*df_Std/np.sqrt(len(df_mean))
            df_upper_99 = df_upper_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper99'})
            df_final = pd.concat([df_mean,df_lower_68,df_upper_68,df_lower_95,df_upper_95,df_lower_99,df_upper_99],axis=1)
            df_final['PLC_CoolingTime_Range'] = df_max['PLC_CoolingTime_Max'] - df_min['PLC_CoolingTime_Min']
            df_final['PLC_InjectionPressure_Range'] = df_max['PLC_InjectionPressure_Max'] - df_min['PLC_InjectionPressure_Min']
            df_final['PLC_MultiTankTemperature_Range'] = df_max['PLC_MultiTankTemperature_Max'] - df_min['PLC_MultiTankTemperature_Min']
            df_final['PLC_InjectionTemperature_Range'] = df_max['PLC_InjectionTemperature_Max'] - df_min['PLC_InjectionTemperature_Min']
            df_final['PLC_NozzleTemperature_Range'] = df_max['PLC_NozzleTemperature_Max'] - df_min['PLC_NozzleTemperature_Min']
        elif machine_number == 14:
            df_mean = total.groupby('index1').mean()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','PassOrFail']]
            df_mean = df_mean.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mean','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mean','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mean','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mean','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mean','PLC_CirculationTank' : 'PLC_CirculationTank_Mean'})
            df_median = total.groupby('index1').median()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_median = df_median.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Median','PLC_InjectionPressure' : 'PLC_InjectionPressure_Median','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Median','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Median','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Median','PLC_CirculationTank' : 'PLC_CirculationTank_Median'})
            df_min = total.groupby('index1').min()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_min = df_min.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Min','PLC_InjectionPressure' : 'PLC_InjectionPressure_Min','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Min','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Min','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Min','PLC_CirculationTank' : 'PLC_CirculationTank_Min'})
            df_max = total.groupby('index1').max()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_max = df_max.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Max','PLC_InjectionPressure' : 'PLC_InjectionPressure_Max','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Max','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Max','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Max','PLC_CirculationTank' : 'PLC_CirculationTank_Max'})
            df_quantile_10 = total.groupby('index1').quantile(0.1)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_10 = df_quantile_10.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.1','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.1','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.1','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.1','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.1','PLC_CirculationTank' : 'PLC_CirculationTank_0.1'})
            df_quantile_25 = total.groupby('index1').quantile(0.25)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_25 = df_quantile_25.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.25','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.25','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.25','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.25','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.25','PLC_CirculationTank' : 'PLC_CirculationTank_0.25'})
            df_quantile_75 = total.groupby('index1').quantile(0.75)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_75 = df_quantile_75.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.75','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.75','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.75','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.75','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.75','PLC_CirculationTank' : 'PLC_CirculationTank_0.75'})
            df_quantile_90 = total.groupby('index1').quantile(0.90)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_90 = df_quantile_90.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.90','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.90','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.90','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.90','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.90','PLC_CirculationTank' : 'PLC_CirculationTank_0.90'})
            df_std = total.groupby('index1').std()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_std = df_std.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Std','PLC_InjectionPressure' : 'PLC_InjectionPressure_Std','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Std','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Std','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Std','PLC_CirculationTank' : 'PLC_CirculationTank_Std'})
            df_mode = total.groupby('index1')[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']].agg(lambda x: x.mode()[0])
            df_mode = df_mode.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mode','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mode','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mode','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mode','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mode','PLC_CirculationTank' : 'PLC_CirculationTank_Mode'})
            df_Std = df_mean
            df_lower_95 = df_mean - 1.96*df_Std/np.sqrt(len(df_mean))
            df_lower_95 = df_lower_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower95'})
            df_upper_95 = df_mean + 1.96*df_Std/np.sqrt(len(df_mean))
            df_upper_95 = df_upper_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper95'})
            df_lower_99 = df_mean - 2.58*df_Std/np.sqrt(len(df_mean))
            df_lower_99 = df_lower_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower99'})
            df_upper_99 = df_mean + 2.58*df_Std/np.sqrt(len(df_mean))
            df_upper_99 = df_upper_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper99'})
            df_final = pd.concat([df_mean,df_median,df_min,df_max,df_quantile_10,df_quantile_25,df_quantile_75,df_quantile_90,df_mode,df_std,df_lower_95,df_upper_95,df_lower_99,df_upper_99],axis=1)
            df_final['PLC_CoolingTime_Range'] = df_max['PLC_CoolingTime_Max'] - df_min['PLC_CoolingTime_Min']
            df_final['PLC_InjectionPressure_Range'] = df_max['PLC_InjectionPressure_Max'] - df_min['PLC_InjectionPressure_Min']
            df_final['PLC_MultiTankTemperature_Range'] = df_max['PLC_MultiTankTemperature_Max'] - df_min['PLC_MultiTankTemperature_Min']
            df_final['PLC_InjectionTemperature_Range'] = df_max['PLC_InjectionTemperature_Max'] - df_min['PLC_InjectionTemperature_Min']
            df_final['PLC_NozzleTemperature_Range'] = df_max['PLC_NozzleTemperature_Max'] - df_min['PLC_NozzleTemperature_Min']
            df_final['PLC_CirculationTank_Range'] = df_max['PLC_CirculationTank_Max'] - df_min['PLC_CirculationTank_Min']
        elif machine_number == 15:
            df_mean = total.groupby('index1').mean()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','PassOrFail']]
            df_mean = df_mean.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mean','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mean','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mean','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mean','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mean','PLC_CirculationTank' : 'PLC_CirculationTank_Mean'})
            df_median = total.groupby('index1').median()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_median = df_median.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Median','PLC_InjectionPressure' : 'PLC_InjectionPressure_Median','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Median','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Median','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Median','PLC_CirculationTank' : 'PLC_CirculationTank_Median'})
            df_min = total.groupby('index1').min()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_min = df_min.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Min','PLC_InjectionPressure' : 'PLC_InjectionPressure_Min','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Min','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Min','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Min','PLC_CirculationTank' : 'PLC_CirculationTank_Min'})
            df_max = total.groupby('index1').max()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_max = df_max.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Max','PLC_InjectionPressure' : 'PLC_InjectionPressure_Max','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Max','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Max','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Max','PLC_CirculationTank' : 'PLC_CirculationTank_Max'})
            df_quantile_10 = total.groupby('index1').quantile(0.1)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_10 = df_quantile_10.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.1','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.1','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.1','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.1','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.1','PLC_CirculationTank' : 'PLC_CirculationTank_0.1'})
            df_quantile_25 = total.groupby('index1').quantile(0.25)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_25 = df_quantile_25.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.25','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.25','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.25','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.25','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.25','PLC_CirculationTank' : 'PLC_CirculationTank_0.25'})
            df_quantile_75 = total.groupby('index1').quantile(0.75)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_75 = df_quantile_75.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.75','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.75','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.75','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.75','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.75','PLC_CirculationTank' : 'PLC_CirculationTank_0.75'})
            df_quantile_90 = total.groupby('index1').quantile(0.90)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_90 = df_quantile_90.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.90','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.90','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.90','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.90','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.90','PLC_CirculationTank' : 'PLC_CirculationTank_0.90'})
            df_std = total.groupby('index1').std()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_std = df_std.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Std','PLC_InjectionPressure' : 'PLC_InjectionPressure_Std','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Std','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Std','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Std','PLC_CirculationTank' : 'PLC_CirculationTank_Std'})
            df_mode = total.groupby('index1')[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']].agg(lambda x: x.mode()[0])
            df_mode = df_mode.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mode','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mode','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mode','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mode','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mode','PLC_CirculationTank' : 'PLC_CirculationTank_Mode'})
            df_final = pd.concat([df_mean,df_median,df_min,df_max,df_quantile_10,df_quantile_25,df_quantile_75,df_quantile_90,df_mode,df_std],axis=1)
            df_final['PLC_CoolingTime_Range'] = df_max['PLC_CoolingTime_Max'] - df_min['PLC_CoolingTime_Min']
            df_final['PLC_InjectionPressure_Range'] = df_max['PLC_InjectionPressure_Max'] - df_min['PLC_InjectionPressure_Min']
            df_final['PLC_MultiTankTemperature_Range'] = df_max['PLC_MultiTankTemperature_Max'] - df_min['PLC_MultiTankTemperature_Min']
            df_final['PLC_InjectionTemperature_Range'] = df_max['PLC_InjectionTemperature_Max'] - df_min['PLC_InjectionTemperature_Min']
            df_final['PLC_NozzleTemperature_Range'] = df_max['PLC_NozzleTemperature_Max'] - df_min['PLC_NozzleTemperature_Min']
            df_final['PLC_CirculationTank_Range'] = df_max['PLC_CirculationTank_Max'] - df_min['PLC_CirculationTank_Min']
        elif machine_number == 16:  
            df_mean = total.groupby('index1').mean()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','PassOrFail']]
            df_mean = df_mean.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mean','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mean','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mean','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mean','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mean','PLC_CirculationTank' : 'PLC_CirculationTank_Mean'})
            df_median = total.groupby('index1').median()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_median = df_median.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Median','PLC_InjectionPressure' : 'PLC_InjectionPressure_Median','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Median','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Median','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Median','PLC_CirculationTank' : 'PLC_CirculationTank_Median'})
            df_min = total.groupby('index1').min()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_min = df_min.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Min','PLC_InjectionPressure' : 'PLC_InjectionPressure_Min','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Min','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Min','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Min','PLC_CirculationTank' : 'PLC_CirculationTank_Min'})
            df_max = total.groupby('index1').max()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_max = df_max.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Max','PLC_InjectionPressure' : 'PLC_InjectionPressure_Max','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Max','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Max','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Max','PLC_CirculationTank' : 'PLC_CirculationTank_Max'})
            df_quantile_10 = total.groupby('index1').quantile(0.1)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_10 = df_quantile_10.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.1','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.1','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.1','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.1','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.1','PLC_CirculationTank' : 'PLC_CirculationTank_0.1'})
            df_quantile_25 = total.groupby('index1').quantile(0.25)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_25 = df_quantile_25.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.25','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.25','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.25','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.25','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.25','PLC_CirculationTank' : 'PLC_CirculationTank_0.25'})
            df_quantile_75 = total.groupby('index1').quantile(0.75)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_75 = df_quantile_75.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.75','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.75','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.75','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.75','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.75','PLC_CirculationTank' : 'PLC_CirculationTank_0.75'})
            df_quantile_90 = total.groupby('index1').quantile(0.90)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_90 = df_quantile_90.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.90','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.90','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.90','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.90','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.90','PLC_CirculationTank' : 'PLC_CirculationTank_0.90'})
            df_std = total.groupby('index1').std()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_std = df_std.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Std','PLC_InjectionPressure' : 'PLC_InjectionPressure_Std','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Std','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Std','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Std','PLC_CirculationTank' : 'PLC_CirculationTank_Std'})
            df_mode = total.groupby('index1')[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']].agg(lambda x: x.mode()[0])
            df_mode = df_mode.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mode','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mode','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mode','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mode','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mode','PLC_CirculationTank' : 'PLC_CirculationTank_Mode'})
            df_Std = df_mean
            df_lower_68 = df_mean - 1.65*df_Std/np.sqrt(len(df_mean))
            df_lower_68 = df_lower_68.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower68','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower68','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower68','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower68','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower68','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower68'})
            df_upper_68 = df_mean + 1.65*df_Std/np.sqrt(len(df_mean))
            df_upper_68 = df_upper_68.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper68','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper68','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper68','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper68','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper68','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper68'})
            df_lower_95 = df_mean - 1.96*df_Std/np.sqrt(len(df_mean))
            df_lower_95 = df_lower_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower95'})
            df_upper_95 = df_mean + 1.96*df_Std/np.sqrt(len(df_mean))
            df_upper_95 = df_upper_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper95'})
            df_lower_99 = df_mean - 2.58*df_Std/np.sqrt(len(df_mean))
            df_lower_99 = df_lower_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower99'})
            df_upper_99 = df_mean + 2.58*df_Std/np.sqrt(len(df_mean))
            df_upper_99 = df_upper_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper99'})
            df_final = pd.concat([df_mean,df_median,df_min,df_max,df_quantile_10,df_quantile_25,df_quantile_75,df_quantile_90,df_mode,df_lower_68,df_upper_68,df_lower_95,df_upper_95,df_lower_99,df_upper_99],axis=1)
            df_final['PLC_CoolingTime_Range'] = df_max['PLC_CoolingTime_Max'] - df_min['PLC_CoolingTime_Min']
            df_final['PLC_InjectionPressure_Range'] = df_max['PLC_InjectionPressure_Max'] - df_min['PLC_InjectionPressure_Min']
            df_final['PLC_MultiTankTemperature_Range'] = df_max['PLC_MultiTankTemperature_Max'] - df_min['PLC_MultiTankTemperature_Min']
            df_final['PLC_InjectionTemperature_Range'] = df_max['PLC_InjectionTemperature_Max'] - df_min['PLC_InjectionTemperature_Min']
            df_final['PLC_NozzleTemperature_Range'] = df_max['PLC_NozzleTemperature_Max'] - df_min['PLC_NozzleTemperature_Min']
            df_final['PLC_CirculationTank_Range'] = df_max['PLC_CirculationTank_Max'] - df_min['PLC_CirculationTank_Min']
        elif machine_number == 17:   
            df_mean = total.groupby('index1').mean()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','PassOrFail']]
            df_mean = df_mean.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mean','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mean','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mean','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mean','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mean','PLC_CirculationTank' : 'PLC_CirculationTank_Mean'})
            df_median = total.groupby('index1').median()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_median = df_median.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Median','PLC_InjectionPressure' : 'PLC_InjectionPressure_Median','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Median','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Median','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Median','PLC_CirculationTank' : 'PLC_CirculationTank_Median'})
            df_min = total.groupby('index1').min()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_min = df_min.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Min','PLC_InjectionPressure' : 'PLC_InjectionPressure_Min','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Min','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Min','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Min','PLC_CirculationTank' : 'PLC_CirculationTank_Min'})
            df_max = total.groupby('index1').max()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_max = df_max.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Max','PLC_InjectionPressure' : 'PLC_InjectionPressure_Max','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Max','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Max','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Max','PLC_CirculationTank' : 'PLC_CirculationTank_Max'})
            df_quantile_10 = total.groupby('index1').quantile(0.1)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_10 = df_quantile_10.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.1','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.1','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.1','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.1','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.1','PLC_CirculationTank' : 'PLC_CirculationTank_0.1'})
            df_quantile_25 = total.groupby('index1').quantile(0.25)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_25 = df_quantile_25.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.25','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.25','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.25','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.25','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.25','PLC_CirculationTank' : 'PLC_CirculationTank_0.25'})
            df_quantile_75 = total.groupby('index1').quantile(0.75)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_75 = df_quantile_75.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.75','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.75','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.75','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.75','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.75','PLC_CirculationTank' : 'PLC_CirculationTank_0.75'})
            df_quantile_90 = total.groupby('index1').quantile(0.90)[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_quantile_90 = df_quantile_90.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_0.90','PLC_InjectionPressure' : 'PLC_InjectionPressure_0.90','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_0.90','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_0.90','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_0.90','PLC_CirculationTank' : 'PLC_CirculationTank_0.90'})
            df_std = total.groupby('index1').std()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_std = df_std.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Std','PLC_InjectionPressure' : 'PLC_InjectionPressure_Std','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Std','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Std','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Std','PLC_CirculationTank' : 'PLC_CirculationTank_Std'})
            df_mode = total.groupby('index1')[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']].agg(lambda x: x.mode()[0])
            df_mode = df_mode.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mode','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mode','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mode','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mode','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mode','PLC_CirculationTank' : 'PLC_CirculationTank_Mode'})
            df_Std = df_mean
            df_lower_68 = df_mean - 1.65*df_Std/np.sqrt(len(df_mean))
            df_lower_68 = df_lower_68.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower68','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower68','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower68','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower68','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower68','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower68'})
            df_upper_68 = df_mean + 1.65*df_Std/np.sqrt(len(df_mean))
            df_upper_68 = df_upper_68.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper68','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper68','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper68','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper68','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper68','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper68'})
            df_lower_95 = df_mean - 1.96*df_Std/np.sqrt(len(df_mean))
            df_lower_95 = df_lower_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower95'})
            df_upper_95 = df_mean + 1.96*df_Std/np.sqrt(len(df_mean))
            df_upper_95 = df_upper_95.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper95','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper95','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper95','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper95','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper95','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper95'})
            df_lower_99 = df_mean - 2.58*df_Std/np.sqrt(len(df_mean))
            df_lower_99 = df_lower_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Lower99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Lower99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Lower99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Lower99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Lower99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Lower99'})
            df_upper_99 = df_mean + 2.58*df_Std/np.sqrt(len(df_mean))
            df_upper_99 = df_upper_99.rename(columns = {'PLC_CoolingTime_Mean' : 'PLC_CoolingTime_Upper99','PLC_InjectionPressure_Mean' : 'PLC_InjectionPressure_Upper99','PLC_MultiTankTemperature_Mean' : 'PLC_MultiTankTemperature_Upper99','PLC_InjectionTemperature_Mean' : 'PLC_InjectionTemperature_Upper99','PLC_NozzleTemperature_Mean' : 'PLC_NozzleTemperature_Upper99','PLC_CirculationTank_Mean' : 'PLC_CirculationTank_Upper99'})
            df_final = pd.concat([df_mean,df_median,df_min,df_max,df_quantile_10,df_quantile_25,df_quantile_75,df_quantile_90,df_mode,df_lower_95,df_upper_95],axis=1)
            df_final['PLC_CoolingTime_Range'] = df_max['PLC_CoolingTime_Max'] - df_min['PLC_CoolingTime_Min']
            df_final['PLC_InjectionPressure_Range'] = df_max['PLC_InjectionPressure_Max'] - df_min['PLC_InjectionPressure_Min']
            df_final['PLC_MultiTankTemperature_Range'] = df_max['PLC_MultiTankTemperature_Max'] - df_min['PLC_MultiTankTemperature_Min']
            df_final['PLC_InjectionTemperature_Range'] = df_max['PLC_InjectionTemperature_Max'] - df_min['PLC_InjectionTemperature_Min']
            df_final['PLC_NozzleTemperature_Range'] = df_max['PLC_NozzleTemperature_Max'] - df_min['PLC_NozzleTemperature_Min']
            df_final['PLC_CirculationTank_Range'] = df_max['PLC_CirculationTank_Max'] - df_min['PLC_CirculationTank_Min'] 
        elif machine_number == 18:
            df_mean = total.groupby('index1').mean()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','PassOrFail']]
            df_mean = df_mean.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mean','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mean','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mean','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mean','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mean','PLC_CirculationTank' : 'PLC_CirculationTank_Mean'})
            df_median = total.groupby('index1').median()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_median = df_median.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Median','PLC_InjectionPressure' : 'PLC_InjectionPressure_Median','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Median','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Median','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Median','PLC_CirculationTank' : 'PLC_CirculationTank_Median'})
            df_min = total.groupby('index1').min()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_min = df_min.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Min','PLC_InjectionPressure' : 'PLC_InjectionPressure_Min','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Min','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Min','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Min','PLC_CirculationTank' : 'PLC_CirculationTank_Min'})
            df_max = total.groupby('index1').max()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_max = df_max.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Max','PLC_InjectionPressure' : 'PLC_InjectionPressure_Max','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Max','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Max','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Max','PLC_CirculationTank' : 'PLC_CirculationTank_Max'})
            df_final = pd.concat([df_mean,df_median,df_min,df_max],axis=1)

        #### 4.) 최종 실시간 추론용 데이터 완성 
        Final_Injection_Mold = total_df
        for col in Final_Injection_Mold.columns:
            try : Final_Injection_Mold[col] = Final_Injection_Mold[col].astype('float')
            except: pass
        if 'FaultyType' in Final_Injection_Mold.columns:
            Final_Injection_Mold = Final_Injection_Mold.drop(['FaultyType'],axis=1)  
        else : 
            pass   
        Final_Injection_Mold = Final_Injection_Mold.rename(columns = {'Value1' : 'WaxTank_Temperature','Value2' : 'Injection_Temperature','Value3' : 'CirculationHose_Temperature','Value4' : 'Nozzle_Temperature'})
        Final_Matched =  df_final.drop(['PassOrFail'],axis=1)  
        Final_Injection_Mold = Final_Injection_Mold.set_index('index1')
        Final_Injection_Mold = Final_Injection_Mold.loc[Final_Matched.index]
        Final = pd.concat([Final_Injection_Mold,Final_Matched ],axis=1)
        test_data = Final.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail'],axis=1)
        Label = Final['PassOrFail'].replace({0:1, 1:0}) 
        return test_data, Label, index_plc 
    
    
#### 3.) 사출기별 학습모델 업로드 (Stacking 모델 선택 )
def prediction_stacking(machine_number, test_data, plc):
    index_plc = preprocessing(test_data, plc)[2] 
    if len(index_plc) == 0:
        machine_number = str(machine_number)
        global file_list, filename
        filename = os.listdir(f'{modelBasePath}/Machine/NonMatched/machine{machine_number}/')
        filename_pkl = []
        for file in filename:
            if file.split('.')[1] == 'pkl':filename_pkl.append(file)
        filename_pkl.remove('Stacking.pkl')
        filename_pkl.sort()
        filelist_df = pd.DataFrame(filename_pkl, columns = ['filename'])
        filelist_df['fileNum']= filelist_df.filename.map(lambda x: x.split('.')[0][3:]).astype('int')
        filelist_df = filelist_df.sort_values(by='fileNum').reset_index(drop=True)
        file_list = list(filelist_df.filename.values)
        test_X = preprocessing(test_data, plc)[0]                                               
        preds = []
        filename = f'{modelBasePath}/Machine/NonMatched/machine{machine_number}/'
        for i in range(len(file_list)):
            model = joblib.load(filename + file_list[i])
            preds.append(model.predict(test_X))
        preds = np.transpose(preds)
        stacking = joblib.load(filename + 'Stacking.pkl')
        result = stacking.predict(preds)
        result = pd.DataFrame(result.T).replace({0:1, 1:0})
        return result
    
    else :
        machine_number = str(machine_number)
        filename = os.listdir(f'{modelBasePath}/Machine/Matched/machine{machine_number}/')
        filename_pkl = []
        for file in filename:
            if file.split('.')[1] == 'pkl':filename_pkl.append(file)
        filename_pkl.remove('Stacking.pkl')
        filename_pkl.sort()
        filelist_df = pd.DataFrame(filename_pkl, columns = ['filename'])
        filelist_df['fileNum']= filelist_df.filename.map(lambda x: x.split('.')[0][3:]).astype('int')
        filelist_df = filelist_df.sort_values(by='fileNum').reset_index(drop=True)
        file_list = list(filelist_df.filename.values)
        test_X = preprocessing(test_data, plc)[0]                                                                           
        preds = []
        filename = f'{modelBasePath}/Machine/Matched/machine{machine_number}/'
        for i in range(len(file_list)):
            model = joblib.load(filename + file_list[i])
            preds.append(model.predict(test_X))
        preds = np.transpose(preds)
        stacking = joblib.load(filename + 'Stacking.pkl')
        result = stacking.predict(preds)
        result = pd.DataFrame(result.T).replace({0:1, 1:0})
        return result

#### 4.) 최종 실시간 예측 함수 작성 
def get_prediction(machine_number,test_data,plc): 
    machine_number=int(machine_number)
    prediction_result = prediction_stacking(machine_number, test_data, plc).iloc[-1] # Recent Value 
    if prediction_result.iloc[-1] == 0.0:
        print('\n')
        print('Prediction Result = Fault')
    else :
        print('\n')
        print(' Prediction Result = Good')
        
    return {"prediction_result":prediction_result.values.tolist()[-1]}  

#### 1.) 실시간 수집할 사출기 번호 & 데이터 개수 key 정의 

In [ ]:
machine_number =18
plc_length = 15000
test_length = 10000

#### 2.) 최근 1개 데이터씩 지도학습 모델 기반 품질 예측 완료 

In [2]:
get_prediction(machine_number=machine_number, test_data=test_data, plc=plc)

10000it [00:02, 3621.02it/s]


Matched Data Exists
Used Matched Injection Mold & PLC Data for Prediction


10000it [00:02, 3692.39it/s]


Matched Data Exists
Used Matched Injection Mold & PLC Data for Prediction


 Prediction Result = Good


{'prediction_result': 1.0}